In [1]:
import cv2
import os
from helpers import load_training_data

In [2]:
import matplotlib.image as mpimg

In [3]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data


def load_training_data(folder_dir, n=100):
    """
    Loads all the training files
    """
    img_dir = folder_dir + "images/"
    files = os.listdir(img_dir)
    n = min(n, len(files))  
    print("Loading " + str(n) + " images")
    imgs = [load_image(img_dir + files[i]) for i in range(n)]

    gnd_dir = folder_dir + "groundtruth/"
    print("Loading " + str(n) + " images")
    gnd_imgs = [load_image(gnd_dir + files[i]) for i in range(n)]
    return imgs, gnd_imgs

In [4]:
# Loading training images and corresponding ground truth
data_dir = '../data/training/'
load_training_data(data_dir)

Loading 100 images


PermissionError: [Errno 13] Permission denied: '../data/training/images/.ipynb_checkpoints'

In [ ]:

#    save_dir = "../data/augmented_training/"
#    dir_imgs = save_dir + "images/"
#    dir_gt = save_dir + "groundtruth/"

# Data Augmentation

augment_all(n, imgs, gt_imgs)



In [7]:
sys.path

['C:\\Users\\salom\\Desktop\\ML\\ml-project-2-magical_mandrills\\scripts',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\python310.zip',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\DLLs',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\lib',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2',
 '',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\lib\\site-packages',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\lib\\site-packages\\win32',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\lib\\site-packages\\win32\\lib',
 'C:\\Users\\salom\\anaconda3\\envs\\mlProject2\\lib\\site-packages\\Pythonwin']